In [1]:
##The feature engineering was redone starting from the original work and with extra work by us.

In [2]:

#updates:
# added magic feature - boost the AUC by 0.0003

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from plotly.offline import iplot, init_notebook_mode

init_notebook_mode(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold

import gc
import os 

#print where those datasets are located 
from sklearn.preprocessing import minmax_scale
# from sklearn.cluster import KMeans

## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


#df_trans = pd.read_csv('train_transaction.csv')
df_trans = pd.read_csv('train_transaction.csv',index_col='TransactionID')
df_test_trans = pd.read_csv('test_transaction.csv',index_col='TransactionID')

#record index of test datasets
test_trans_id = df_test_trans.index

#df_id = pd.read_csv('train_identity.csv')
df_id = pd.read_csv('train_identity.csv',index_col='TransactionID')
df_test_id = pd.read_csv('test_identity.csv',index_col='TransactionID')




sample_submission = pd.read_csv('sample_submission.csv', index_col='TransactionID')


In [3]:
def id_split(dataframe):
    
    #expand = split and expand to seperate column
    dataframe['device_name'] = dataframe['DeviceInfo'].str.split('/', expand=True)[0]
    dataframe['device_version'] = dataframe['DeviceInfo'].str.split('/', expand=True)[1]

    dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
    dataframe['version_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[1]

    dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
    dataframe['version_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[1]

    dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
    dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]

    dataframe['id_34'] = dataframe['id_34'].str.split(':', expand=True)[1]
    dataframe['id_23'] = dataframe['id_23'].str.split(':', expand=True)[1]
    
    #very specifically group device brands to help making decision
    dataframe.loc[dataframe['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    dataframe.loc[dataframe['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    dataframe.loc[dataframe['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    dataframe.loc[dataframe['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    dataframe.loc[dataframe['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    dataframe.loc[dataframe['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    #make the device name that are less than 200 to be other
    dataframe.loc[dataframe.device_name.isin(dataframe.device_name.value_counts()[dataframe.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    
    #new column to record that this device has id???
    dataframe['had_id'] = 1
    gc.collect()
    
    return dataframe

In [4]:
df_trans[df_trans['V326']==0]['isFraud']

TransactionID
2987004    0
2987008    0
2987016    0
2987017    0
2987022    0
          ..
3577465    0
3577495    0
3577506    0
3577526    1
3577531    0
Name: isFraud, Length: 72668, dtype: int64

In [5]:

#C sum feature
df_trans["C_sum"] = df_trans.loc[:,'C1':'C14'].sum(axis = 1).astype(np.int8)
df_test_trans["C_sum"] = df_test_trans.loc[:,'C1':'C14'].sum(axis = 1).astype(np.int8)

    
##D1 - D15
df_trans['D_na'] =df_trans.loc[:,'D1':'D14'].isna().sum(axis=1).astype(np.int8)
df_test_trans['D_na'] =df_test_trans.loc[:,'D1':'D14'].isna().sum(axis=1).astype(np.int8)

    
##M1-9
i_cols = ['M1','M2','M3','M5','M6','M7','M8','M9']

df_trans['M_na'] = df_trans[i_cols].isna().sum(axis=1).astype(np.int8)
df_test_trans['M_na'] = df_test_trans[i_cols].isna().sum(axis=1).astype(np.int8)


    
##V_ features
df_trans['V_na'] =df_trans.loc[:,"V1":"V339"].isna().sum(axis=1).astype(np.int8)
df_test_trans['V_na'] =df_test_trans.loc[:,"V1":"V339"].isna().sum(axis=1).astype(np.int8)

##ids 
df_id['id_na'] = df_id.loc[:,"id_01":"id_38"].isna().sum(axis=1).astype(np.int8)
df_test_id['id_na'] = df_test_id.loc[:,"id_01":"id_38"].isna().sum(axis=1).astype(np.int8)




In [6]:
df_id = id_split(df_id)
df_test_id = id_split(df_test_id)

In [7]:

##do not need to include right_index
df_train = df_trans.merge(df_id, how='left', left_index=True,  on='TransactionID')
df_test = df_test_trans.merge(df_test_id, how='left', left_index=True, on='TransactionID')

#add total na for train and test data. 
df_train['tot_na'] = df_train.isna().sum(axis=1).astype(np.int8) 
df_test['tot_na'] = df_test.isna().sum(axis=1).astype(np.int8)

print(df_train.head())
print(df_test.head())


# y_train = df_train['isFraud'].copy()
del df_trans, df_id, df_test_trans, df_test_id
gc.collect()



df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)
##Deal with missing data



               isFraud  TransactionDT  TransactionAmt ProductCD  card1  card2  \
TransactionID                                                                   
2987000              0          86400            68.5         W  13926    NaN   
2987001              0          86401            29.0         W   2755  404.0   
2987002              0          86469            59.0         W   4663  490.0   
2987003              0          86499            50.0         W  18132  567.0   
2987004              0          86506            50.0         H   4497  514.0   

               card3       card4  card5   card6  ...  device_name  \
TransactionID                                    ...                
2987000        150.0    discover  142.0  credit  ...          NaN   
2987001        150.0  mastercard  102.0  credit  ...          NaN   
2987002        150.0        visa  166.0   debit  ...          NaN   
2987003        150.0  mastercard  117.0   debit  ...          NaN   
2987004        150

In [8]:
df_train.loc[3577531,"screen_height"]

'1600'

In [9]:
#https://www.kaggle.com/davidcairuz/feature-engineering-lightgbm#
#removed V300 feature
useful_features = ['TransactionDT','TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
                   'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13',
                   'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M2', 'M3',
                   'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V17',
                   'V19', 'V20', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48',
                   'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71',
                   'V72', 'V73', 'V74', 'V75', 'V76', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92',
                   'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140',
                   'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161',
                   'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V170', 'V171', 'V172', 'V173', 'V175', 'V176', 'V177',
                   'V178', 'V180', 'V182', 'V184', 'V187', 'V188', 'V189', 'V195', 'V197', 'V200', 'V201', 'V202', 'V203', 'V204',
                   'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V219', 'V220',
                   'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V231', 'V233', 'V234', 'V238', 'V239',
                   'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V249', 'V251', 'V253', 'V256', 'V257', 'V258', 'V259', 'V261',
                   'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276',
                   'V277', 'V278', 'V279', 'V280', 'V282', 'V283', 'V285', 'V287', 'V288', 'V289', 'V291', 'V292', 'V294','id_01',
                   'id_02', 'id_03', 'id_05', 'id_06', 'id_09',
                   'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31', 'id_32', 'id_33',
                   'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'device_name', 'device_version', 'OS_id_30', 'version_id_30',
                   'browser_id_31', 'version_id_31', 'screen_width', 'screen_height', 'had_id','tot_na','C_sum','D_na','M_na','V_na','id_na']
print(len(useful_features))

273


In [10]:
cols_to_drop = [col for col in df_train.columns if col not in useful_features]
cols_to_drop.remove('isFraud')


In [11]:
df_train = df_train.drop(cols_to_drop, axis=1)
df_test = df_test.drop(cols_to_drop, axis=1)

In [12]:
df_train

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,device_name,device_version,OS_id_30,version_id_30,browser_id_31,version_id_31,screen_width,screen_height,had_id,tot_na
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.500000,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-12
2987001,0,86401,29.000000,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-16
2987002,0,86469,59.000000,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-35
2987003,0,86499,50.000000,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-19
2987004,0,86506,50.000000,H,4497,514.0,150.0,mastercard,102.0,credit,...,Samsung,NRD90M,Android,7.0,samsung,browser,2220,1080,1.0,-119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577535,0,15811047,49.000000,W,6550,NaN,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-16
3577536,0,15811049,39.500000,W,10444,225.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-34
3577537,0,15811079,30.953125,W,12037,595.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29


In [13]:
print(df_train)
print(df_test)

               isFraud  TransactionDT  TransactionAmt ProductCD  card1  card2  \
TransactionID                                                                   
2987000              0          86400       68.500000         W  13926    NaN   
2987001              0          86401       29.000000         W   2755  404.0   
2987002              0          86469       59.000000         W   4663  490.0   
2987003              0          86499       50.000000         W  18132  567.0   
2987004              0          86506       50.000000         H   4497  514.0   
...                ...            ...             ...       ...    ...    ...   
3577535              0       15811047       49.000000         W   6550    NaN   
3577536              0       15811049       39.500000         W  10444  225.0   
3577537              0       15811079       30.953125         W  12037  595.0   
3577538              0       15811088      117.000000         W   7826  481.0   
3577539              0      

In [14]:
#new feature regarding TransactionDT
# any repeated transation consecutively under one card1 number is very suspicious. 
df_train= df_train.sort_values(by =['card1','TransactionDT'])

rpt_trans = [1  if df_train.iloc[i,2] == df_train.iloc[i+1,2] or df_train.iloc[i,2] == df_train.iloc[i-1,2] else 0 for i in range(1,df_train.shape[0]-1)]
df_train['rept_trans'] =[0]+ rpt_trans +[0]

df_test= df_test.sort_values(by =['card1','TransactionDT'])

rpt_trans = [1  if df_test.iloc[i,2] == df_test.iloc[i+1,2] or df_test.iloc[i,2] == df_test.iloc[i-1,2] else 0 for i in range(1,df_test.shape[0]-1)]
df_test['rept_trans'] =[0]+ rpt_trans +[0]

In [15]:
#added card6 - debit credit
columns_a = ['TransactionAmt', 'id_02', 'D15']
columns_b = ['card1', 'card2']

for col_a in columns_a:
    for col_b in columns_b:
        for df in [df_train, df_test]:
            df[f'{col_a}_to_mean_{col_b}'] = df[col_a] / df.groupby([col_b])[col_a].transform('mean')
            df[f'{col_a}_to_std_{col_b}'] = df[col_a] / df.groupby([col_b])[col_a].transform('std')

In [16]:
# New feature - log of transaction amount.
df_train['TransactionAmt_Log'] = np.log(df_train['TransactionAmt'])
df_test['TransactionAmt_Log'] = np.log(df_test['TransactionAmt'])

# New feature - decimal part of the transaction amount.
df_train['TransactionAmt_decimal'] = ((df_train['TransactionAmt'] - df_train['TransactionAmt'].astype(int)) * 1000).astype(int)
df_test['TransactionAmt_decimal'] = ((df_test['TransactionAmt'] - df_test['TransactionAmt'].astype(int)) * 1000).astype(int)

# New feature - day of week in which a transaction happened.
df_train['Transaction_day_of_week'] = np.floor((df_train['TransactionDT'] / (3600 * 24) - 1) % 7)
df_test['Transaction_day_of_week'] = np.floor((df_test['TransactionDT'] / (3600 * 24) - 1) % 7)

# New feature - hour of the day in which a transaction happened.
df_train['Transaction_hour'] = np.floor(df_train['TransactionDT'] / 3600) % 24
df_test['Transaction_hour'] = np.floor(df_test['TransactionDT'] / 3600) % 24

# Some arbitrary features interaction
for feature in ['id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain',
                'card2__dist1', 'card1__card5', 'card2__id_20', 'card1__P_emaildomain', 'addr1__card1']:

    f1, f2 = feature.split('__')
    df_train[feature] = df_train[f1].astype(str) + '_' + df_train[f2].astype(str)
    df_test[feature] = df_test[f1].astype(str) + '_' + df_test[f2].astype(str)

    le = preprocessing.LabelEncoder()
    le.fit(list(df_train[feature].astype(str).values) + list(df_test[feature].astype(str).values))
    df_train[feature] = le.transform(list(df_train[feature].astype(str).values))
    df_test[feature] = le.transform(list(df_test[feature].astype(str).values))

# Encoding - count encoding for both df_train and df_test
for feature in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'id_36']:
    df_train[feature + '_count_full'] = df_train[feature].map(pd.concat([df_train[feature], df_test[feature]], ignore_index=True).value_counts(dropna=False))
    df_test[feature + '_count_full'] = df_test[feature].map(pd.concat([df_train[feature], df_test[feature]], ignore_index=True).value_counts(dropna=False))

# Encoding - count encoding separately for df_train and df_test
for feature in ['id_01', 'id_31', 'id_33', 'id_36']:
    df_train[feature + '_count_dist'] = df_train[feature].map(df_train[feature].value_counts(dropna=False))
    df_test[feature + '_count_dist'] = df_test[feature].map(df_test[feature].value_counts(dropna=False))

In [17]:
df_train = df_train.reset_index()
df_train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,card2_count_full,card3_count_full,card4_count_full,card5_count_full,card6_count_full,id_36_count_full,id_01_count_dist,id_31_count_dist,id_33_count_dist,id_36_count_dist
0,3230924,0,5787419,23.4375,C,1000,555.0,185.0,mastercard,224.0,...,80404,109960,347386,153109,824959,267353,82170,5806,517251,134066
1,3023634,0,916268,183.0000,W,1001,555.0,150.0,visa,226.0,...,80404,956845,719649,553537,824959,819269,446307,450258,517251,449555
2,3151336,0,3504180,29.0000,W,1001,555.0,150.0,visa,226.0,...,80404,956845,719649,553537,824959,819269,446307,450258,517251,449555
3,3210739,0,5270458,27.0000,W,1001,555.0,150.0,visa,226.0,...,80404,956845,719649,553537,824959,819269,446307,450258,517251,449555
4,3020767,0,842821,150.0000,R,1004,583.0,150.0,visa,226.0,...,41503,956845,719649,553537,267648,267353,82170,22000,263,134066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3419314,0,10938204,232.5000,W,18395,543.0,150.0,mastercard,224.0,...,9968,956845,347386,153109,824959,819269,446307,450258,517251,449555
590536,3420746,0,10964952,36.9375,W,18395,543.0,150.0,mastercard,224.0,...,9968,956845,347386,153109,824959,819269,446307,450258,517251,449555
590537,3446622,0,11806909,36.9375,W,18395,543.0,150.0,mastercard,224.0,...,9968,956845,347386,153109,824959,819269,446307,450258,517251,449555
590538,3480204,0,12865537,36.9375,W,18395,543.0,150.0,mastercard,224.0,...,9968,956845,347386,153109,824959,819269,446307,450258,517251,449555


In [18]:


#k fold cross validation target mean coding to avoid overfitting

N_SPLITS = 5
SEED = 6

folds = KFold(n_splits=N_SPLITS,shuffle = False, random_state=SEED)
# Test Data and expport DF
  
score = 0
y_result = 0

mean_of_target = df_train['isFraud'].mean()

target_mean_list =['card1','card2','addr1','addr1__card1','card1__card5','card1__P_emaildomain']

# kfold target mean encoding for regularisation

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train)):
    print('Fold:',fold_+1)
    tr_x = df_train.iloc[trn_idx,:] 
    vl_x = df_train.iloc[val_idx,:]  
    
    for col in target_mean_list:
        print(col)
        df_train.loc[val_idx,f'{col}_target_enc'] = vl_x[col].map(tr_x.groupby(col)['isFraud'].mean())
        
        df_train[f'{col}_target_enc'].fillna(mean_of_target, inplace = True)
                                                        
for col in target_mean_list:
                                                            
    df_test[f'{col}_target_enc'] = df_test[col].map(df_train.groupby(col)[f'{col}_target_enc'].mean())

    df_test[f'{col}_target_enc'].fillna(mean_of_target, inplace = True)

    
    

Fold: 1
card1
card2
addr1
addr1__card1
card1__card5
card1__P_emaildomain
Fold: 2
card1
card2
addr1
addr1__card1
card1__card5
card1__P_emaildomain
Fold: 3
card1
card2
addr1
addr1__card1
card1__card5
card1__P_emaildomain
Fold: 4
card1
card2
addr1
addr1__card1
card1__card5
card1__P_emaildomain
Fold: 5
card1
card2
addr1
addr1__card1
card1__card5
card1__P_emaildomain


In [19]:
#label encoding the rest 
for col in df_train.columns:
    if df_train[col].dtype == 'object':
        le = preprocessing.LabelEncoder()
        le.fit(list(df_train[col].astype(str).values) + list(df_test[col].astype(str).values))
        df_train[col] = le.transform(list(df_train[col].astype(str).values))
        df_test[col] = le.transform(list(df_test[col].astype(str).values))

In [20]:
#why sort ??
X_train = df_train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT','TransactionID'], axis=1)
y_train = df_train.sort_values('TransactionDT')['isFraud']

X_test = df_test.drop(['TransactionDT'], axis=1)

del df_train, df_test
gc.collect()

20

In [21]:
X_test

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,...,id_01_count_dist,id_31_count_dist,id_33_count_dist,id_36_count_dist,card1_target_enc,card2_target_enc,addr1_target_enc,addr1__card1_target_enc,card1__card5_target_enc,card1__P_emaildomain_target_enc
TransactionID,,,,,,,,,,,,,,,,,,,,,
4084529,50.0000,1,1001,555.0,150.0,4,226.0,2,269.0,87.0,...,82579,16054,6441,133287,0.03499,0.022705,0.035629,0.03499,0.03499,0.03499
4113026,39.0000,4,1001,555.0,150.0,4,226.0,2,269.0,87.0,...,364784,370066,436020,369714,0.03499,0.022705,0.035629,0.03499,0.03499,0.03499
4147003,60.0000,1,1002,NaN,150.0,2,NaN,1,343.0,87.0,...,82579,13098,2746,133287,0.03499,0.034990,0.017703,0.03499,0.03499,0.03499
3981622,42.9375,4,1003,452.0,150.0,2,224.0,2,264.0,87.0,...,364784,370066,436020,369714,0.03499,0.000650,0.018267,0.03499,0.03499,0.03499
4044940,100.0000,1,1004,583.0,150.0,4,226.0,1,325.0,87.0,...,12000,362,1555,3690,0.03499,0.041566,0.025488,0.03499,0.03499,0.03499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004795,117.0000,4,18396,111.0,150.0,4,226.0,2,126.0,87.0,...,364784,370066,436020,369714,0.03499,0.021892,0.025796,0.03499,0.03499,0.03499
4054400,226.0000,4,18396,111.0,150.0,4,226.0,2,126.0,87.0,...,364784,370066,436020,369714,0.03499,0.021892,0.025796,0.03499,0.03499,0.03499
4072739,226.0000,4,18396,111.0,150.0,4,226.0,2,126.0,87.0,...,364784,370066,436020,369714,0.03499,0.021892,0.025796,0.03499,0.03499,0.03499


In [22]:
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

##replace inf vlaues 



In [23]:
X_train[X_train> np.finfo(np.float32).max] = 10000
X_test[X_test> np.finfo(np.float32).max] = 10000

In [24]:
np.where(X_train.values >= np.finfo(np.float64).max)

(array([], dtype=int64), array([], dtype=int64))

In [25]:
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier

from sklearn.model_selection import KFold, train_test_split
from sklearn import metrics





In [26]:
clf1 = LGBMClassifier(
                    objective='binary',
                    n_estimators = 2000,
                    boosting_type='gbdt',
                    metric='auc',
                    n_jobs=-1,
                    learning_rate=0.01,
                    num_leaves= 2**8,
                    max_depth=-1,
                    colsample_bytree= 0.7,
                    subsample_freq=1,
                    subsample=0.7,
                    max_bin=255,
                    verbose=4,
                    seed= SEED,
                    random_state=1, 
                    silent = False,
                    )

clf2 = XGBClassifier(
                    n_estimators=1000,
                    max_depth=10, 
                    verbose = 10, 
                    learning_rate=0.07, 
                    subsample=0.8, 
                    colsample_bytree=0.9,
                    missing=-999,
                    random_state=1, 
                    objective = 'binary:logistic', 
                    tree_method='hist')
    
clf3 = AdaBoostClassifier(
                    algorithm='SAMME.R', 
                    base_estimator=None,
                    learning_rate=0.07, 
                    n_estimators=1000, 
                    random_state=1
                    )

clf4 = GradientBoostingClassifier(
                    n_estimators=1000,
                    min_samples_leaf = 100,
                    verbose = 1,
                    min_weight_fraction_leaf=0.0345, 
                    max_depth=10, 
                    max_features=0.7, 
                    learning_rate=0.07, 
                    random_state=1
                    )
                    
clf5 = CatBoostClassifier(
                    n_estimators=1000,
                    verbose = 1,
                    max_depth=10, 
                    random_state=1,
                    learning_rate=0.07, 
                    class_weights = [1,28.58]
                    )

In [27]:
ensemble_class = VotingClassifier(estimators=[
         ('lgb', clf1), ('xgb', clf2), ('adb', clf3),('gb',clf4),('cb',clf5)],
         voting='soft')
##when using weight = [4,1,1,1,1], LB = 0.938

In [28]:

print("Fitting....")
ensemble_model = ensemble_class.fit(X_train, y_train)


Fitting....
      Iter       Train Loss   Remaining Time 
         1           0.2928          192.57m
         2           0.2849          184.80m
         3           0.2793          182.90m
         4           0.2738          182.50m
         5           0.2695          179.63m
         6           0.2653          180.09m
         7           0.2621          179.76m
         8           0.2590          180.31m
         9           0.2563          180.44m
        10           0.2540          179.33m
        20           0.2368          177.63m
        30           0.2282          174.44m
        40           0.2182          172.80m
        50           0.2114          170.12m
        60           0.2065          167.43m
        70           0.2026          164.13m
        80           0.1991          162.97m
        90           0.1961          160.88m
       100           0.1934          158.65m
       200           0.1776          136.79m
       300           0.1687          118.8

130:	learn: 0.2922259	total: 2m 40s	remaining: 17m 44s
131:	learn: 0.2918072	total: 2m 41s	remaining: 17m 42s
132:	learn: 0.2911988	total: 2m 42s	remaining: 17m 41s
133:	learn: 0.2905183	total: 2m 44s	remaining: 17m 40s
134:	learn: 0.2895056	total: 2m 45s	remaining: 17m 39s
135:	learn: 0.2891422	total: 2m 46s	remaining: 17m 38s
136:	learn: 0.2885326	total: 2m 47s	remaining: 17m 37s
137:	learn: 0.2882017	total: 2m 49s	remaining: 17m 36s
138:	learn: 0.2876512	total: 2m 50s	remaining: 17m 34s
139:	learn: 0.2866660	total: 2m 51s	remaining: 17m 33s
140:	learn: 0.2862017	total: 2m 52s	remaining: 17m 32s
141:	learn: 0.2852022	total: 2m 54s	remaining: 17m 31s
142:	learn: 0.2849846	total: 2m 55s	remaining: 17m 30s
143:	learn: 0.2846008	total: 2m 56s	remaining: 17m 29s
144:	learn: 0.2840604	total: 2m 57s	remaining: 17m 28s
145:	learn: 0.2837630	total: 2m 59s	remaining: 17m 27s
146:	learn: 0.2833113	total: 3m	remaining: 17m 26s
147:	learn: 0.2820093	total: 3m 1s	remaining: 17m 25s
148:	learn: 0.2

280:	learn: 0.2009988	total: 5m 46s	remaining: 14m 45s
281:	learn: 0.2003592	total: 5m 47s	remaining: 14m 44s
282:	learn: 0.1999738	total: 5m 48s	remaining: 14m 42s
283:	learn: 0.1994839	total: 5m 49s	remaining: 14m 41s
284:	learn: 0.1988748	total: 5m 50s	remaining: 14m 40s
285:	learn: 0.1982447	total: 5m 52s	remaining: 14m 39s
286:	learn: 0.1973128	total: 5m 53s	remaining: 14m 38s
287:	learn: 0.1968470	total: 5m 54s	remaining: 14m 36s
288:	learn: 0.1964824	total: 5m 55s	remaining: 14m 35s
289:	learn: 0.1960180	total: 5m 57s	remaining: 14m 34s
290:	learn: 0.1955284	total: 5m 58s	remaining: 14m 33s
291:	learn: 0.1950132	total: 5m 59s	remaining: 14m 32s
292:	learn: 0.1947228	total: 6m	remaining: 14m 31s
293:	learn: 0.1940994	total: 6m 2s	remaining: 14m 29s
294:	learn: 0.1938203	total: 6m 3s	remaining: 14m 28s
295:	learn: 0.1933471	total: 6m 4s	remaining: 14m 27s
296:	learn: 0.1928772	total: 6m 5s	remaining: 14m 26s
297:	learn: 0.1926710	total: 6m 7s	remaining: 14m 25s
298:	learn: 0.19219

431:	learn: 0.1481868	total: 8m 53s	remaining: 11m 41s
432:	learn: 0.1478924	total: 8m 54s	remaining: 11m 40s
433:	learn: 0.1473530	total: 8m 56s	remaining: 11m 39s
434:	learn: 0.1470443	total: 8m 57s	remaining: 11m 37s
435:	learn: 0.1467825	total: 8m 58s	remaining: 11m 36s
436:	learn: 0.1463144	total: 8m 59s	remaining: 11m 35s
437:	learn: 0.1461814	total: 9m	remaining: 11m 34s
438:	learn: 0.1457351	total: 9m 2s	remaining: 11m 32s
439:	learn: 0.1454344	total: 9m 3s	remaining: 11m 31s
440:	learn: 0.1452836	total: 9m 4s	remaining: 11m 30s
441:	learn: 0.1449803	total: 9m 5s	remaining: 11m 29s
442:	learn: 0.1448440	total: 9m 7s	remaining: 11m 28s
443:	learn: 0.1443570	total: 9m 8s	remaining: 11m 26s
444:	learn: 0.1440445	total: 9m 9s	remaining: 11m 25s
445:	learn: 0.1438969	total: 9m 10s	remaining: 11m 24s
446:	learn: 0.1433332	total: 9m 12s	remaining: 11m 23s
447:	learn: 0.1430738	total: 9m 13s	remaining: 11m 21s
448:	learn: 0.1427028	total: 9m 14s	remaining: 11m 20s
449:	learn: 0.1425370

581:	learn: 0.1146184	total: 11m 59s	remaining: 8m 36s
582:	learn: 0.1143645	total: 12m	remaining: 8m 35s
583:	learn: 0.1141988	total: 12m 1s	remaining: 8m 34s
584:	learn: 0.1139458	total: 12m 3s	remaining: 8m 33s
585:	learn: 0.1137387	total: 12m 4s	remaining: 8m 31s
586:	learn: 0.1133842	total: 12m 5s	remaining: 8m 30s
587:	learn: 0.1132346	total: 12m 6s	remaining: 8m 29s
588:	learn: 0.1130335	total: 12m 8s	remaining: 8m 28s
589:	learn: 0.1128357	total: 12m 9s	remaining: 8m 26s
590:	learn: 0.1126893	total: 12m 10s	remaining: 8m 25s
591:	learn: 0.1123263	total: 12m 11s	remaining: 8m 24s
592:	learn: 0.1122285	total: 12m 13s	remaining: 8m 23s
593:	learn: 0.1120963	total: 12m 14s	remaining: 8m 21s
594:	learn: 0.1120323	total: 12m 15s	remaining: 8m 20s
595:	learn: 0.1118481	total: 12m 16s	remaining: 8m 19s
596:	learn: 0.1115327	total: 12m 18s	remaining: 8m 18s
597:	learn: 0.1114356	total: 12m 19s	remaining: 8m 17s
598:	learn: 0.1112415	total: 12m 20s	remaining: 8m 15s
599:	learn: 0.1109468

732:	learn: 0.0906841	total: 15m 6s	remaining: 5m 30s
733:	learn: 0.0906094	total: 15m 8s	remaining: 5m 29s
734:	learn: 0.0904617	total: 15m 9s	remaining: 5m 27s
735:	learn: 0.0903493	total: 15m 10s	remaining: 5m 26s
736:	learn: 0.0902358	total: 15m 11s	remaining: 5m 25s
737:	learn: 0.0900825	total: 15m 13s	remaining: 5m 24s
738:	learn: 0.0900152	total: 15m 14s	remaining: 5m 22s
739:	learn: 0.0897873	total: 15m 15s	remaining: 5m 21s
740:	learn: 0.0894725	total: 15m 16s	remaining: 5m 20s
741:	learn: 0.0894008	total: 15m 18s	remaining: 5m 19s
742:	learn: 0.0892989	total: 15m 19s	remaining: 5m 17s
743:	learn: 0.0891631	total: 15m 20s	remaining: 5m 16s
744:	learn: 0.0890135	total: 15m 21s	remaining: 5m 15s
745:	learn: 0.0887925	total: 15m 22s	remaining: 5m 14s
746:	learn: 0.0887100	total: 15m 24s	remaining: 5m 13s
747:	learn: 0.0886528	total: 15m 25s	remaining: 5m 11s
748:	learn: 0.0885570	total: 15m 26s	remaining: 5m 10s
749:	learn: 0.0884369	total: 15m 27s	remaining: 5m 9s
750:	learn: 0.

883:	learn: 0.0739752	total: 18m 13s	remaining: 2m 23s
884:	learn: 0.0738562	total: 18m 15s	remaining: 2m 22s
885:	learn: 0.0736855	total: 18m 16s	remaining: 2m 21s
886:	learn: 0.0736078	total: 18m 17s	remaining: 2m 19s
887:	learn: 0.0735544	total: 18m 18s	remaining: 2m 18s
888:	learn: 0.0733983	total: 18m 20s	remaining: 2m 17s
889:	learn: 0.0732702	total: 18m 21s	remaining: 2m 16s
890:	learn: 0.0730648	total: 18m 22s	remaining: 2m 14s
891:	learn: 0.0729999	total: 18m 23s	remaining: 2m 13s
892:	learn: 0.0729255	total: 18m 25s	remaining: 2m 12s
893:	learn: 0.0728374	total: 18m 26s	remaining: 2m 11s
894:	learn: 0.0726950	total: 18m 27s	remaining: 2m 9s
895:	learn: 0.0725645	total: 18m 28s	remaining: 2m 8s
896:	learn: 0.0725134	total: 18m 30s	remaining: 2m 7s
897:	learn: 0.0724127	total: 18m 31s	remaining: 2m 6s
898:	learn: 0.0723284	total: 18m 32s	remaining: 2m 4s
899:	learn: 0.0721702	total: 18m 33s	remaining: 2m 3s
900:	learn: 0.0721192	total: 18m 35s	remaining: 2m 2s
901:	learn: 0.072

In [29]:
##trick here 
##for oof = 0.93547, LB score is 0.9458
##for oof = 0.936, LB score is 0.945

In [34]:
   
y_result    = ensemble_model.predict_proba(X_test)


In [35]:

sub = pd.DataFrame(columns=['TransactionID','isFraud'])
sub['TransactionID'] = X_test.index
sub['isFraud'] = y_result[:,1]
sub




sub.to_csv("submission59th-updated02-.csv",index=False)

sum(y_result[:,1]>0.5)


8671

In [33]:
sum(y_result[:,1]>0.1)

317512